In [1]:
import datetime, os, json, csv
from collections import Counter

In [2]:
nowtime = f'{datetime.datetime.now():%Y%m%d}'
one_week_ago = f'{datetime.datetime.now() - datetime.timedelta(days=7):%Y%m%d}'
print(nowtime)

20210927


In [3]:
os.makedirs('./resources/ch09-twittercookbook/'+ str(nowtime), exist_ok = True)

## API 設定

In [4]:
import twitter

def oauth_login():

    CONSUMER_KEY = 'KOThudX3C7brplSrDa7vSVM2T' #API key 
    CONSUMER_SECRET = 'Fx5shLni0RcK1mPvLkANVbzSmGYK3pUIM0ZfiebpdzCykvVNuA' #API key select
    OAUTH_TOKEN = '109162821-Kh4y6qfclwVBKr8cz978DU7Fz1F0nGs7HiZH34PV'
    OAUTH_TOKEN_SECRET = 'rXd5DKwiAiKQJIBY6o7vH66Od5mQXrHEBP1GEVqoBY9bE'
    
    auth = twitter.oauth.OAuth(OAUTH_TOKEN, OAUTH_TOKEN_SECRET,
                               CONSUMER_KEY, CONSUMER_SECRET)
    
    twitter_api = twitter.Twitter(auth=auth)
    return twitter_api

# Sample usage
twitter_api = oauth_login()    

# Nothing to see by displaying twitter_api except that it's now a
# defined variable

print(twitter_api)

## ツィートを検索する

In [5]:
from urllib.parse import unquote

when = str(#20200903 #20200908 #20200915 
    #20200917 20200924 
    nowtime
)
# XXX: Set this variable to a trending topic, 
# or anything else for that matter. The example query below
# was a trending topic when this content was being developed
# and is used throughout the remainder of this chapter.

#q = '出生前診断' #11 2020/9/3　 word 448　9/8 483　9/15 600　9/17 600 9/24 492 9/27 365
#q = 'NIPT' #35  word 427 9/8  358 9/15 368 9/17 343 9/24 310　9/27 306

#q = 'クアトロテスト' #0  word 18 9/8 15 9/15 14 9/17 12 9/24 18 9/27-10/8 14 
#q = '羊水検査' #9/27 = 113 
#q= 'コンバインドテスト' #9/27 5　# 
#q= 'トリプルマーカーテスト' #9/27 1 10/13 1
#q= 'AYA世代' #9/27 1 10/13 1 11/24 408
q = '産婦人科'

count = 100

# See https://dev.twitter.com/docs/api/1.1/get/search/tweets

search_results = twitter_api.search.tweets(q=q, count=count,lang='ja')

statuses = search_results['statuses']


# Iterate through 5 more batches of results by following the curkeywosor

for _ in range(5):
    print("Length of statuses", len(statuses))
    try:
        next_results = search_results['search_metadata']['next_results']
    except KeyError as e: # No more results when next_results doesn't exist
        break
        
    # Create a dictionary from next_results, which has the following form:
    # ?max_id=313519052523986943&q=NCAA&include_entities=1
    kwargs = dict([ kv.split('=') for kv in unquote(next_results[1:]).split("&") ])
    
    search_results = twitter_api.search.tweets(**kwargs)
    statuses += search_results['statuses']

# Show one sample search result by slicing the list...
#print(json.dumps(statuses[0], indent=1,ensure_ascii=False))
print(one_week_ago,'から',when,'までの、キーワード「',
      q,'」を含むツィートの件数',len(statuses))

Length of statuses 100
Length of statuses 200
Length of statuses 300
Length of statuses 400
Length of statuses 500
20210920 から 20210927 までの、キーワード「 産婦人科 」を含むツィートの件数 600


## ツィートを検索する（関数）

In [6]:
def twitter_search(twitter_api, q, max_results=100, **kw):

    # See https://dev.twitter.com/docs/api/1.1/get/search/tweets and 
    # https://dev.twitter.com/docs/using-search for details on advanced 
    # search criteria that may be useful for keyword arguments
    
    # See https://dev.twitter.com/docs/api/1.1/get/search/tweets    
    search_results = twitter_api.search.tweets(q=q, count=100, **kw)
    
    statuses = search_results['statuses']
    
    # Iterate through batches of results by following the cursor until we
    # reach the desired number of results, keeping in mind that OAuth users
    # can "only" make 180 search queries per 15-minute interval. See
    # https://dev.twitter.com/docs/rate-limiting/1.1/limits
    # for details. A reasonable number of results is ~1000, although
    # that number of results may not exist for all queries.
    
    # Enforce a reasonable limit
    max_results = min(1000, max_results)
    
    for _ in range(10): # 10*100 = 1000
        try:
            next_results = search_results['search_metadata']['next_results']
        except KeyError as e: # No more results when next_results doesn't exist
            break
            
        # Create a dictionary from next_results, which has the following form:
        # ?max_id=313519052523986943&q=NCAA&include_entities=1
        kwargs = dict([ kv.split('=') 
                        for kv in next_results[1:].split("&") ])
        
        search_results = twitter_api.search.tweets(**kwargs)
        statuses += search_results['statuses']
        
        if len(statuses) > max_results: 
            break
            
    return statuses

# Sample usage

twitter_api = oauth_login()

#q = "出生前診断"
results = twitter_search(twitter_api, q, max_results=10,lang='ja')
        
# Show one sample search result by slicing the list...
print(json.dumps(results[0], indent=1,ensure_ascii=False))
#print(len(statuses))

{
 "created_at": "Mon Sep 27 02:34:49 +0000 2021",
 "id": 1442316768437960704,
 "id_str": "1442316768437960704",
 "text": "RT @renatkhsh: 親ブロックとは例えば産婦人科の場合、娘が『ピルを飲みたい』『HPVワクチンを打ちたい』『産婦人科にかかりたい』時に親が阻害する事である。\n実は配偶者ブロックというものもあり、癌と診断され標準治療を勧められるも本人の意思に反して配偶者が抗癌剤や手術な…",
 "truncated": false,
 "entities": {
  "hashtags": [],
  "symbols": [],
  "user_mentions": [
   {
    "screen_name": "renatkhsh",
    "name": "高橋怜奈/産婦人科医YouTuber",
    "id": 258788622,
    "id_str": "258788622",
    "indices": [
     3,
     13
    ]
   }
  ],
  "urls": []
 },
 "metadata": {
  "iso_language_code": "ja",
  "result_type": "recent"
 },
 "source": "<a href=\"http://twitter.com/download/android\" rel=\"nofollow\">Twitter for Android</a>",
 "in_reply_to_status_id": null,
 "in_reply_to_status_id_str": null,
 "in_reply_to_user_id": null,
 "in_reply_to_user_id_str": null,
 "in_reply_to_screen_name": null,
 "user": {
  "id": 3295761,
  "id_str": "3295761",
  "name": "ǝunsʇo ıɯnɟɐsɐɯ",
  "screen_name": "otsune",
  "loc

## JSONデータをテキストで保存する

In [7]:
import io, json

In [8]:
def save_json(filename, data):
    with io.open('resources/ch09-twittercookbook/{0}/{1}.json'.format(nowtime,filename), 
                 'w', encoding='utf-8') as f:
        f.write(json.dumps(data, ensure_ascii=False))

def load_json(filename):
    with io.open('resources/ch09-twittercookbook/{0}/{1}.json'.format(nowtime,filename), 
                 encoding='utf-8') as f:
        return f.read()

# Sample usage

#q = '出生前診断'

twitter_api = oauth_login()
results = twitter_search(twitter_api, q, max_results=10,lang='ja')

save_json(q, results)
#results = load_json(q)
#print(json.dumps(results, indent=1,ensure_ascii=False))

## 保存したJSONを開く

In [9]:
statuses = json.loads(
    open('resources/ch09-twittercookbook/'+when +'/'+str(q) + '.json').read())


## テキスト、ハッシュタグ、screen namesを抽出する

In [10]:
status_texts = [ status['text'] 
                 for status in statuses ]

screen_names = [ user_mention['screen_name'] 
                 for status in statuses
                     for user_mention in status['entities']['user_mentions'] ]

hashtags = [ hashtag['text'] 
             for status in statuses
                 for hashtag in status['entities']['hashtags'] ]

# Compute a collection of all words from all tweets
words = [ w 
          for t in status_texts 
              for w in t.split() ]

# Explore the first 5 items for each...
#print('texts: {}\n screen_names: {}\n hashtags: {}\n words: {}'.format(
    #len(statuses),len(screen_names),len(hashtags),len(words)))
print(json.dumps(status_texts[0:5], indent=1,ensure_ascii=False))
print(json.dumps(screen_names[0:5], indent=1,ensure_ascii=False))
print(json.dumps(hashtags[0:5], indent=1,ensure_ascii=False))
#print(json.dumps(words[0:5], indent=1,ensure_ascii=False))

[
 "RT @renatkhsh: 親ブロックとは例えば産婦人科の場合、娘が『ピルを飲みたい』『HPVワクチンを打ちたい』『産婦人科にかかりたい』時に親が阻害する事である。\n実は配偶者ブロックというものもあり、癌と診断され標準治療を勧められるも本人の意思に反して配偶者が抗癌剤や手術な…",
 "RT @fujitatakanori: 出産したくない、中絶費用が支払えない、という相談も多くあるけれど、費用は後でどうにでもなるから、まずは産婦人科とその医療相談室に相談してほしい。医師、福祉職はあなたを責めないし事情を考慮して対処する。\n無料低額診療の届出がある病院もおすす…",
 "ドイツ人産婦人科医が世界で初めてグレーフェンベルク・リングこと子宮内避妊具を発明し、世界中に広まった★日本で認可されたのは1974年★気に入ったらリツイート！",
 "RT @fujitatakanori: 出産したくない、中絶費用が支払えない、という相談も多くあるけれど、費用は後でどうにでもなるから、まずは産婦人科とその医療相談室に相談してほしい。医師、福祉職はあなたを責めないし事情を考慮して対処する。\n無料低額診療の届出がある病院もおすす…",
 "@you_yu_sub 目安だけどね(*•̀ᴗ•́*)👍\nあとは産婦人科さんとかの何ヶ月健診とかで相談？聞くのもありだよ(◍˃ ᵕ ˂◍) https://t.co/A8y1RJw5rW"
]
[
 "renatkhsh",
 "fujitatakanori",
 "fujitatakanori",
 "you_yu_sub",
 "fujitatakanori"
]
[
 "妊娠初期",
 "自然妊娠",
 "流産",
 "初期流産",
 "2018年"
]


## 単語をCSV

In [11]:
l = []
for label in words:
    #l.append('\n')
    l.append(label)
    #l.append('\n')
    with open('./result_csv/'+str(q)+'_result_'+when+'.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerows([l])

## 結果をテーブル表示

In [12]:
from prettytable import PrettyTable

for label, data in (('Word', words), 
                    ('Screen Name', screen_names), 
                    ('Hashtag', hashtags)):
    pt = PrettyTable(field_names=[label, 'Count']) 
    c = Counter(data)
    [ pt.add_row(kv) for kv in c.most_common()[:]]
    pt.align[label], pt.align['Count'] = 'l', 'r' # Set column alignment
    print(pt)
    #print(c.most_common())

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
| Word                                                                                                                                                                                                                                                                                   | Count |
+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------+
| RT                                                                                                                           

# Mecabのインストール
https://qiita.com/ekzemplaro/items/c98c7f6698f130b55d53


##### 